In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
DATAFILE = "BDP.csv"
df = pd.read_csv(DATAFILE)

# drop the column that has only NA values
df = df.drop(['ADJ_SG&A_TO_SALES'], axis=1)

# now we want to remove the rows that have a lot
# of NA values
# NOTE: I am also going to clean the ratings column here
# as I iterate through the data (remove the asterisks)
remove = []
threshold = 0.5
for index, row in df.iterrows():
    # print(index)
    
    # there are some cells that say "#N/A Field Not Applicable"
    # change these to just be regular NA values
    for key, value in row.items():
        if value == "#N/A Field Not Applicable":
            df.at[index, key] = np.nan
    
    cur_row = row.isnull()
    if cur_row['RTG_SP_LT_LC_ISSUER_CREDIT']:
        # print(row['Ticker'])
        remove.append(index)
    else:
        total_count = 0
        na_count = 0
        for bools in cur_row.items():
            total_count += 1
            if bools[1]:
                na_count += 1
        if na_count/total_count > threshold:
            # print(row['Ticker'])
            remove.append(index)
        else:
            # if it reaches here then it means the row is good
            # we will now remove asterisks if they appear in the rating
            cleaned_rating = re.findall(r'[^\s]*', row['RTG_SP_LT_LC_ISSUER_CREDIT'])[0]
            if cleaned_rating == "BBBpi":
                cleaned_rating = "BBB"
            df.at[index, 'RTG_SP_LT_LC_ISSUER_CREDIT'] = cleaned_rating
            
# print(remove)
# print(len(remove))
df = df.drop(df.index[remove])
# print(abc.shape)

In [4]:
# I want to take care of the NA and blank values here
# if the feature is categorical I will replace blanks with "MISSING"
# if the feature is continuous I will replace with the mean of the column

# first need to convert the data type of the continuous variables into floats
all_columns = set(df.columns)
categorical = set(['Ticker', 'Rating Date', 'Fiscal Year', 'RTG_SP_LT_LC_ISSUER_CREDIT', 'COUNTRY', 
                    'GICS_INDUSTRY_NAME'])

continuous = list(all_columns - categorical)

df[continuous] = df[continuous].apply(pd.to_numeric)
# the continuous variables are now float variables

# lets fix the categorical variables first
for column in list(categorical):
    df[column] = df[column].fillna('MISSING')
    
for column in continuous:
    df[column] = df[column].fillna((df[column].mean()))

In [5]:
# return clean csv so we can just load the clean dataset
df.to_csv(DATAFILE.split('.')[0] + '_CLEAN.csv', index=False)